In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
fopen = open('../dataset/digit/train.csv', 'r')
train_df = pd.read_csv(fopen)
fopen.close()
fopen = open('../dataset/digit/test.csv', 'r')
test_df = pd.read_csv(fopen)
fopen.close()

In [3]:
train_df.info()
print '----------------------'
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB
----------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [4]:
train_label = train_df.label.values
train_df.drop(['label'], inplace=True, axis=1)

train_arr = train_df.values.astype(np.float32)
test_arr = test_df.values.astype(np.float32)

In [5]:
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_arr, train_label, test_size=0.05, random_state=42)
n_sample = X_train.shape[0]
y_train = label_binarize(y_train, range(10))
y_test = label_binarize(y_test, range(10))

In [6]:
def add_fc_layer(input, input_size, output_size, activation=None):
    input = tf.reshape(input, [-1, input_size]) 
    w = tf.Variable(tf.random_normal([input_size, output_size], dtype=tf.float32))
    b = tf.Variable(tf.random_normal([1, output_size], dtype=tf.float32))
    out = tf.matmul(input, w) + b
    return activation(out) if activation else out

In [7]:
def add_conv_layer(x, filter_size, activation=None):
    w = tf.Variable(tf.random_normal(filter_size), dtype=tf.float32)
    conv = tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='SAME')
    b = tf.Variable(tf.random_normal([1, filter_size[-1]], dtype=tf.float32))
    out = conv + b
    return activation(out) if activation else out

def add_pool_layer(x, activation=None):
    pool = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    return activation(pool) if activation else pool

In [8]:
def acc(pred, target):
    return (np.argmax(pred, axis=1) == np.argmax(target, axis=1)).sum() / float(target.shape[0])

In [9]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
reshape = tf.reshape(x, [-1, 28, 28, 1])
keep_prob = tf.placeholder(tf.float32)

In [10]:
l1 = add_conv_layer(reshape, [5, 5, 1, 32], tf.nn.relu)
l2 = add_pool_layer(l1)

l3 = add_conv_layer(l2, [5, 5, 32, 64], tf.nn.relu)
l4 = add_pool_layer(l3)

l5 = add_fc_layer(l4, 7*7*64, 1024, tf.nn.relu)
drop = tf.nn.dropout(l5, keep_prob=keep_prob)
l6 = add_fc_layer(drop, 1024, 10)

soft_max = tf.nn.softmax(l6)

In [11]:
loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(l6, y))

In [12]:
opt = tf.train.AdamOptimizer(2e-3).minimize(loss)

In [13]:
ss = tf.InteractiveSession()
ss.run(tf.initialize_all_variables())
for step in xrange(20):
    start = step * 100 % X_train.shape[0]
    end = (step * 100 + 100) % X_train.shape[0]
    xs, ys = X_train[start:end], y_train[start:end]
    ss.run(opt, feed_dict={x : xs, y : ys, keep_prob : 0.5})
    if step % 50 == 0:
        train_result = ss.run(soft_max, feed_dict={x : xs, keep_prob : 1})
        test_result = ss.run(soft_max, feed_dict={x : X_test, keep_prob : 1})
        ls = ss.run(loss, feed_dict={x : xs, y : ys, keep_prob : 1})
        print "train_acc={0}, loss={1}, valid_acc={2}".format(acc(train_result, ys), ls, acc(test_result, y_test))
    

train_acc=0.18, loss=1127768832.0, valid_acc=0.169047619048


In [19]:
predict = np.array([])
for i in xrange(test_arr.shape[0] / 1000):
    res = ss.run(soft_max, feed_dict={x : test_arr[i * 1000: i * 1000 + 1000], keep_prob : 1})
    predict = np.append(predict, np.argmax(res, axis=1))

In [24]:
a = np.array([[1,2], [2, 3]])

array([[1, 2],
       [2, 3]])